In [ ]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2
import time
import csv
import queue
from datetime import datetime
import math
import numpy as np

In [ ]:
model_path = "/Users/beni/Desktop/Climbing Technique Tracker/Climbing_Tracker/pose_landmarker_full.task"

In [ ]:
"""The code in this block is partly taken from the official MediaPipe documentation and can be found 
here: https://ai.google.dev/edge/mediapipe/solutions/vision/pose_landmarker/python#live-stream_1"""

BaseOptions = mp.tasks.BaseOptions
PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions
PoseLandmarkerResult = mp.tasks.vision.PoseLandmarkerResult
VisionRunningMode = mp.tasks.vision.RunningMode

saved_results = queue.Queue()

def print_result(result: PoseLandmarkerResult, output_image: mp.Image, timestamp_ms: int):
    print('pose landmarker result: {}'.format(result))
    saved_results.put(result)
    saved_results.put(timestamp_ms)

options = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.LIVE_STREAM, #sets the running mode to live stream
    result_callback=print_result)


"""The following code is written by myself using the OpenCV and MediaPipe documentation for help which
can be found in the readme file"""

#check how many frames per second Qsense sensors output
frames_per_second = 10
time_between_frames = 1 / frames_per_second

with PoseLandmarker.create_from_options(options) as landmarker:
    #Use openCV VideoCapture to start capturing from the webcam
    start_time = time.time()

    capture = cv2.VideoCapture(0)

    #Create a loop to read the latest frame from the camera using VideoCaptureRead()
    while capture.isOpened():

        ret, frame = capture.read()
     
        if not ret:
            print("broken")
            break

        #Convert the frame receieved from OpenCV to a MediaPipe Image Object
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
      
        landmarker.detect_async(mp_image, int(datetime.now().timestamp() * 1000))

        cv2.imshow('frame', frame) #first param is window name
        cv2.waitKey(10)

        """The following frame limiter logic was taken from the following online answer:
         https://www.quora.com/How-do-I-decrease-the-frames-per-second-in-OpenCV-python"""
        
        elapsed_time = time.time() - start_time 
        time_to_wait = time_between_frames - elapsed_time 
        if time_to_wait > 0: 
            time.sleep(time_to_wait)
    

In [ ]:
def calculate_elbow_angle(shoulder_landmark, elbow_landmark, wrist_landmark):

    #convert landmark coordinates into vectors
    #calculate vector without z value to improve accuracy
    vector_s_e = [shoulder_landmark.x - elbow_landmark.x, shoulder_landmark.y - elbow_landmark.y, shoulder_landmark.z - elbow_landmark.z]
    vector_w_e = [wrist_landmark.x - elbow_landmark.x, wrist_landmark.y - elbow_landmark.y, wrist_landmark.z - elbow_landmark.z]
   
    """dot product of the two vectors: code adapted from 
    https://www.geeksforgeeks.org/python/how-to-calculate-dot-product-of-two-vectors-in-python/""" 
    dot_product = np.dot(vector_s_e, vector_w_e)
    

    """magnitude of the two vectors: code apated from
    https://stackoverflow.com/questions/9171158/how-do-you-get-the-magnitude-of-a-vector-in-numpy"""
    magnitude_vector_s_e = np.linalg.norm(vector_s_e)
    magnitude_vector_w_e = np.linalg.norm(vector_w_e)
    

    #final equation to calculate angle
    angle = math.degrees(math.acos(dot_product / ((magnitude_vector_s_e) * (magnitude_vector_w_e))))
    
    return angle

In [ ]:
def midpoint_two_points(landmark_one, landmark_two):
    return [(landmark_one.x + landmark_two.x)/2, 
            (landmark_one.y + landmark_two.x)/2]

In [ ]:
def midpoint_four_points(landmark_one, landmark_two, landmark_three, landmark_four):
    centre_1 = [(landmark_one.x + landmark_two.x)/2, (landmark_one.y + landmark_two.x)/2]
    centre_2 = [(centre_1[0] + landmark_three.x)/2, (centre_1[1] + landmark_three.y)/2]
    centre_3 = [(centre_2[0] + landmark_four.x)/2, (centre_2[1] + landmark_four.y)/2]

    return centre_3

In [ ]:
def calculate_centre_of_mass(landmark):

    CoM_scaled_x_value_women = 0
    CoM_scaled_y_value_women = 0
    CoM_scaled_x_value_men = 0
    CoM_scaled_y_value_men = 0

    head_centre = [landmark.pose_world_landmarks[0][0].x, landmark.pose_world_landmarks[0][0].y]
    head_centre_scaled_women = [head_centre[0]*6.7, head_centre[1]*6.7]
    CoM_scaled_x_value_women += head_centre_scaled_women[0]
    CoM_scaled_y_value_women += head_centre_scaled_women[1]
    head_centre_scaled_men = [head_centre[0]*6.7, head_centre[1]*6.7]
    CoM_scaled_x_value_men += head_centre_scaled_men[0]
    CoM_scaled_y_value_men += head_centre_scaled_men[1]
    
    left_upper_arm_centre = midpoint_two_points(landmark.pose_world_landmarks[0][11], landmark.pose_world_landmarks[0][13])
    left_upper_arm_scaled_women = [left_upper_arm_centre[0] * 2.3, left_upper_arm_centre[1]*2.3]
    CoM_scaled_x_value_women += left_upper_arm_scaled_women[0]
    CoM_scaled_y_value_women += left_upper_arm_scaled_women[1]
    left_upper_arm_scaled_men = [left_upper_arm_centre[0] * 2.4, left_upper_arm_centre[1]*2.4]
    CoM_scaled_x_value_men += left_upper_arm_scaled_men[0]
    CoM_scaled_y_value_men += left_upper_arm_scaled_men[1]

    right_upper_arm_centre = midpoint_two_points(landmark.pose_world_landmarks[0][12], landmark.pose_world_landmarks[0][14])
    right_upper_arm_scaled_women = [right_upper_arm_centre[0] * 2.3, right_upper_arm_centre[1]*2.3]
    CoM_scaled_x_value_women += right_upper_arm_scaled_women[0]
    CoM_scaled_y_value_women += right_upper_arm_scaled_women[1]
    right_upper_arm_scaled_men = [right_upper_arm_centre[0] * 2.4, right_upper_arm_centre[1]*2.4]
    CoM_scaled_x_value_men += right_upper_arm_scaled_men[0]
    CoM_scaled_y_value_men += right_upper_arm_scaled_men[1]

    left_lower_arm_centre = midpoint_two_points(landmark.pose_world_landmarks[0][13], landmark.pose_world_landmarks[0][15])
    left_lower_arm_scaled_women = [left_lower_arm_centre[0]*1.4, left_lower_arm_centre[1]*1.4]
    CoM_scaled_x_value_women += left_lower_arm_scaled_women[0]
    CoM_scaled_y_value_women += left_lower_arm_scaled_women[1]
    left_lower_arm_scaled_men = [left_lower_arm_centre[0]*1.7, left_lower_arm_centre[1]*1.7]
    CoM_scaled_x_value_men += left_lower_arm_scaled_men[0]
    CoM_scaled_y_value_men += left_lower_arm_scaled_men[1]

    right_lower_arm_centre = midpoint_two_points(landmark.pose_world_landmarks[0][14], landmark.pose_world_landmarks[0][16])
    right_lower_arm_scaled_women = [right_lower_arm_centre[0]*1.4, right_lower_arm_centre[1]*1.4]
    CoM_scaled_x_value_women += right_lower_arm_scaled_women[0]
    CoM_scaled_y_value_women += right_lower_arm_scaled_women[1]
    right_lower_arm_scaled_men = [right_lower_arm_centre[0]*1.7, right_lower_arm_centre[1]*1.7]
    CoM_scaled_x_value_men += right_lower_arm_scaled_men[0]
    CoM_scaled_y_value_men += right_lower_arm_scaled_men[1]

    left_hand_centre = midpoint_four_points(landmark.pose_world_landmarks[0][15], landmark.pose_world_landmarks[0][21], landmark.pose_world_landmarks[0][19], landmark.pose_world_landmarks[0][17])
    left_hand_scaled_women = [left_hand_centre[0]* 0.5, left_hand_centre[1]*0.5]
    CoM_scaled_x_value_women += left_hand_scaled_women[0]
    CoM_scaled_y_value_women += left_hand_scaled_women[1]
    left_hand_scaled_men = [left_hand_centre[0]* 0.6, left_hand_centre[1]*0.6]
    CoM_scaled_x_value_men += left_hand_scaled_men[0]
    CoM_scaled_y_value_men += left_hand_scaled_men[1]

    right_hand_centre = midpoint_four_points(landmark.pose_world_landmarks[0][16], landmark.pose_world_landmarks[0][22], landmark.pose_world_landmarks[0][20], landmark.pose_world_landmarks[0][18])
    right_hand_scaled_women = [right_hand_centre[0]* 0.5, right_hand_centre[1]*0.5]
    CoM_scaled_x_value_women += right_hand_scaled_women[0]
    CoM_scaled_y_value_women += right_hand_scaled_women[1]
    right_hand_scaled_men = [right_hand_centre[0]* 0.6, right_hand_centre[1]*0.6]
    CoM_scaled_x_value_men += right_hand_scaled_men[0]
    CoM_scaled_y_value_men += right_hand_scaled_men[1]

    torso_centre = midpoint_four_points(landmark.pose_world_landmarks[0][12], landmark.pose_world_landmarks[0][11], landmark.pose_world_landmarks[0][24], landmark.pose_world_landmarks[0][23])
    torso_centre_scaled_women = [torso_centre[0]*60.8, torso_centre[1]*60.8]
    CoM_scaled_x_value_women += torso_centre_scaled_women[0]
    CoM_scaled_y_value_women += torso_centre_scaled_women[1]
    torso_centre_scaled_men = [torso_centre[0]*66.6, torso_centre[1]*66.6]
    CoM_scaled_x_value_men += torso_centre_scaled_men[0]
    CoM_scaled_y_value_men += torso_centre_scaled_men[1]

    pelvis_centre = midpoint_two_points(landmark.pose_world_landmarks[0][24], landmark.pose_world_landmarks[0][23])
    pelvis_centre_scaled_women = [pelvis_centre[0]* 14.7, pelvis_centre[1]*14.7]
    CoM_scaled_x_value_women += pelvis_centre_scaled_women[0]
    CoM_scaled_y_value_women += pelvis_centre_scaled_women[1]
    pelvis_centre_scaled_men = [pelvis_centre[0]*14.2, pelvis_centre[1]*14.2]
    CoM_scaled_x_value_men += pelvis_centre_scaled_men[0]
    CoM_scaled_y_value_men += pelvis_centre_scaled_men[1]

    left_upper_thigh_centre = midpoint_two_points(landmark.pose_world_landmarks[0][23], landmark.pose_world_landmarks[0][25])
    left_upper_thigh_scaled_women = [left_upper_thigh_centre[0]* 14.6, left_upper_thigh_centre[1]*14.6]
    CoM_scaled_x_value_women += left_upper_thigh_scaled_women[0]
    CoM_scaled_y_value_women += left_upper_thigh_scaled_women[1]
    left_upper_thigh_scaled_men = [left_upper_thigh_centre[0]*12.3, left_upper_arm_centre[1]*12.3]
    CoM_scaled_x_value_men += left_upper_thigh_scaled_men[0]
    CoM_scaled_y_value_men += left_upper_thigh_scaled_men[1]

    right_upper_thigh_centre = midpoint_two_points(landmark.pose_world_landmarks[0][24], landmark.pose_world_landmarks[0][26])
    right_upper_thigh_scaled_women = [right_upper_thigh_centre[0]* 14.6, right_upper_thigh_centre[1]*14.6]
    CoM_scaled_x_value_women += right_upper_thigh_scaled_women[0]
    CoM_scaled_y_value_women += right_upper_thigh_scaled_women[1]
    right_upper_thigh_scaled_men = [right_upper_thigh_centre[0]* 12.3, right_upper_thigh_centre[1]*12.3]
    CoM_scaled_x_value_men += right_upper_thigh_scaled_men[0]
    CoM_scaled_y_value_men += right_upper_thigh_scaled_men[1]

    left_lower_thigh_centre = midpoint_two_points(landmark.pose_world_landmarks[0][25], landmark.pose_world_landmarks[0][27])
    left_lower_thigh_scaled_women = [left_lower_thigh_centre[0]*4.5, left_lower_thigh_centre[1]*4.5]
    CoM_scaled_x_value_women += left_lower_thigh_scaled_women[0]
    CoM_scaled_y_value_women += left_lower_thigh_scaled_women[1]
    left_lower_thigh_scaled_men = [left_lower_thigh_centre[0]*4.8, left_lower_thigh_centre[1]*4.8]
    CoM_scaled_x_value_men += left_lower_thigh_scaled_men[0]
    CoM_scaled_y_value_men += left_lower_thigh_scaled_men[1]

    right_lower_thigh_centre = midpoint_two_points(landmark.pose_world_landmarks[0][26], landmark.pose_world_landmarks[0][28])
    right_lower_thigh_scaled_women = [right_lower_thigh_centre[0]*4.5, right_lower_thigh_centre[1]*4.5]
    CoM_scaled_x_value_women += right_lower_thigh_scaled_women[0]
    CoM_scaled_y_value_women += right_lower_thigh_scaled_women[1]
    right_lower_thigh_scaled_men = [right_lower_thigh_centre[0]*4.8, right_lower_thigh_centre[1]*4.8]
    CoM_scaled_x_value_men += right_lower_thigh_scaled_men[0]
    CoM_scaled_y_value_men += right_lower_thigh_scaled_men[1]

    
    CoM_scaled_women = [CoM_scaled_x_value_women/128.8, CoM_scaled_y_value_women/128.8] #divide by body mass % total to normalize
    CoM_scaled_men = [CoM_scaled_x_value_men/131.1, CoM_scaled_y_value_men/131.1]

    return CoM_scaled_women, CoM_scaled_men

In [ ]:
#adding column headers to the array
results = [
    ['time stamp', 'landmark #0', 'landmark #1', 'landmark #2', 'landmark #3', 'landmark #4', 'landmark #5',
    'landmark #6', 'landmark #7', 'landmark #8', 'landmark #9', 'landmark #10', 'landmark #11',
    'landmark #12', 'landmark #13', 'landmark #14', 'landmark #15', 'landmark #16', 'landmark #17',
    'landmark #18', 'landmark #19', 'landmark #20', 'landmark #21', 'landmark #22', 'landmark #23',
    'landmark #24', 'landmark #25', 'landmark #26', 'landmark #27', 'landmark #28', 'landmark #29',
    'landmark #30','landmark #31','landmark #32', 'world landmark #0', 'world landmark #1', 'world landmark #2', 
    'world landmark #3', 'world landmark #4', 'world landmark #5', 'world landmark #6', 
    'world landmark #7', 'world landmark #8', 'world landmark #9', 'world landmark #10', 
    'world landmark #11', 'world landmark #12', 'world landmark #13', 'world landmark #14',
    'world landmark #15', 'world landmark #16', 'world landmark #17', 'world landmark #18', 
    'world landmark #19', 'world landmark #20', 'world landmark #21', 'world landmark #22', 
    'world landmark #23', 'world landmark #24','world landmark #25', 'world landmark #26', 
    'world landmark #27', 'world landmark #28', 'world landmark #29', 'world landmark #30', 
    'world landmark #31', 'world landmark #32', 'elbow angle', 'Centre of mass women', 
    'Centre of mass men']
]


while not saved_results.empty():
    
    landmark = saved_results.get() #get pose_landmark for each frame
    
    try:
        landmarks = [] #initialize an array for each indidual landmark in a frame (33 total)
        world_landmarks = []
        for i in range(0,33):
            landmark_point = landmark.pose_landmarks[0][i] #get each landmark (0, 32) for the 0th person in the frame
            world_landmark_point = landmark.pose_world_landmarks[0][i]
            landmarks.append(landmark_point)
            world_landmarks.append(world_landmark_point)
    except: #if person out of frame
        saved_results.get() #pop the timestamp so frame and timestamps stay in sync
        continue #skip rest of the loop

    #calculate elbow angle
    try:
        elbow_ang = calculate_elbow_angle(landmark.pose_world_landmarks[0][12], landmark.pose_world_landmarks[0][14], landmark.pose_world_landmarks[0][16])
    except:
        pass
    
    #calculate Centre of Mass
    try:
        CoM = calculate_centre_of_mass(landmark)
    except:
        pass

    from datetime import datetime, timedelta
    time_stamp = saved_results.get() #get timestamp (comes after each landmark in the queue)
    utc_tz = datetime.utcfromtimestamp(time_stamp / 1000) #convert to readable date time format
    utc_plus_2_tz = utc_tz + timedelta(hours=1) #adjust time zone from UTC to UTC + 8
    
    try:
        results.append([utc_plus_2_tz]+ landmarks + world_landmarks+ [elbow_ang] + [CoM[0]] + [CoM[1]]) #append landmarks + timestamp into results array
        print(elbow_ang)
    except:
         results.append([utc_plus_2_tz])


"""CSV code adapted from the following website: https://www.geeksforgeeks.org/python/writing-csv-files-in-python/"""

#each frame is writen into the CSV, each column are the coordinates for a single landmark
with open('results.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(results)